In [1]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import requests
import random
import pickle
import re
import spacy
nlp = spacy.load('en_core_web_trf')
from spacy import displacy
import collections
from collections import Counter
from tqdm.notebook import tqdm as tqdm_notebook
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from transformers import DistilBertTokenizer, DistilBertModel, logging
from matplotlib.figure import Figure
from matplotlib import cm
import matplotlib.colors as colors
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

logging.set_verbosity_error()

In [2]:
import sys
sys.path.insert(0, '../../src/models/')
sys.path.insert(0, '../../src/features/')

import predict_model
from predict_model import loadBERT
from predict_model import SpanPredictor as classify
from build_features import text_cleaner
%matplotlib inline

In [3]:
model = loadBERT("../../models/", 'saved_weights_inf_FIXED_boot_beta80.pt')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

CPU Success


In [4]:
# URL
URL = 'https://en.wikipedia.org/wiki/Glossary_of_bird_terms'
# Get the page
page = requests.get(URL, timeout=5)
soup = BeautifulSoup(page.content, "html.parser", from_encoding="iso-8859-1")   
# Find embedded glossary
glossaries = soup.find_all('dt', {'class': 'glossary'})
parts = [part.text.lower().strip() for part in glossaries]
# Get additional anchors ("also know as...")
glossaries_other = soup.find_all('span', {'class': 'anchor'})
parts_other = [part['id'].lower().strip() for part in glossaries_other]
# Append and drop duplicates
parts = list(set((parts + parts_other)))
# Replace underscore with space
glossary = [part.replace('_', ' ') for part in parts]


# A few helpers
additions = [
    'legs',
    'beak',
    'head',
    'wingspan',
    'eye',
    'forecrown',
    'hindneck',
]

glossary += additions



#with open('../../data/glossaries/birds.pkl', 'wb') as f:
#    pickle.dump(glossary, f)      

In [49]:
# Compound helpers
compounds = [
    'upper', 'lower', 'greater', 'dorsal', 'alternate', 'lesser',
    'central', 'outermost', 'outer', 'inner', 'uppermost', 'median',  
    'upperwing', 'underwing', 'tail', 'rump',
    'secondary', 'primary', 'definitive', 'basic', 'tertial', 'preformative', 'prebasic', 'tertials',
    'posterior', 'fresh', 
    'sexual', 'juvenile', 'male', 'female', 'adult', 'juvenal', 'individual',
    'breeding', 'predefinitive', 'non-breeding',
    'raised', 'closed', 'open', 
    'entire', 'partial',
    

]

In [50]:
# Read all html files
species_folder = glob.glob('../../data/raw/BOW/*')
species_ALL = [link[19:].lower() for link in species_folder]

In [51]:
#species_ALL[0]

In [64]:
def compound_reconstructor(token, doc):
    if token.i == 0:
        trait = token
    elif doc[token.i - 1].pos_ == 'DET':
        trait = token
    elif doc[token.i - 3].dep_ == 'compound':
        trait = doc[token.i - 3: token.i + 1]
    elif doc[token.i - 3].text.lower() in compounds or doc[token.i - 3].lemma_.lower() in compounds:
        trait = doc[token.i - 3: token.i + 1]
    elif doc[token.i - 2].dep_ == 'compound':
        trait = doc[token.i - 2: token.i + 1]
    elif doc[token.i - 2].text.lower() in compounds or doc[token.i - 3].lemma_.lower() in compounds:
        trait = doc[token.i - 2: token.i + 1]
    elif doc[token.i - 1].dep_ == 'compound':
        trait = doc[token.i - 1: token.i + 1]
    elif doc[token.i - 1].text.lower() in compounds or doc[token.i - 3].lemma_.lower() in compounds:
        trait = doc[token.i - 1: token.i + 1]
    else:
        trait = token   
    return trait.lemma_

def check_existance(t, doc, glossary):
    if t.lemma_ in glossary:
        return t
    elif t.text in glossary:
        return t
    else:
        return None

def extract_advmod(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'advmod':
            return child
        
def extract_nummod(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'nummod':
            return child

def extract_conjunction(t, doc):
    """HELPER"""
    if t.dep_ == 'conj' and t.pos_ == 'ADJ':
        return t 

def extract_amod(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'amod':
            return child
        
def extract_measurements(t, doc):
    """HELPER"""
    obj = None
    measurements = ['wide', 'long', 'high',]
    if t.text in measurements or t.lemma_ in measurements:
        obj = doc[t.left_edge.i : t.right_edge.i + 1]
    return obj

def extract_prepositions(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'prep':
            return doc[child.left_edge.i : child.right_edge.i + 1]

def define_position(x, y, doc):
    """HELPER"""
    if len(x.text.split()) > 1:
        return f'{y.text} {x.text}'
    else:
        try:
            if x.i > y.i:
                return doc[y.i : x.i + 1]
            else:
                return doc[x.i : y.i + 1]
        except:
            return f'{y.text} {x.text}'


def extract_verb_advm(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'advmod':
            return child

def extract_verb_nmod(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'nummod':
            return doc[child.left_edge.i : child.right_edge.i + 1] 
        
def extract_verb_prep(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'prep':
            return child    

def extract_verb_pobj(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'pobj' or child.dep_ == 'pcomp' or child.dep_ == 'prep':
            return child
        
def extract_verb_nmod(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'acl':
            return doc[child.left_edge.i : child.right_edge.i + 1]         
        
def extract_verb_dobj(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'dobj':
            return child
        
def extract_verb_orpd(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'oprd':
            return child    
        
def extract_verb_agnt(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'agent':
            return child 
        
def extract_verb_acomp(t, doc):
    """HELPER"""
    for child in t.children:
        if child.dep_ == 'acomp':
            obj = doc[child.left_edge.i : child.right_edge.i + 1].text.lower()
            return obj 
        
def extract_nounandverb_nummods(t, doc):
    obj = None
    for child in t.children:
        if child.dep_ == 'nummod':
            obj = doc[child.left_edge.i : child.right_edge.i + 1]
            return obj   

def extract_dnummod(t, doc):
    obj = extract_nounandverb_nummods(t, doc)
    if obj:
        return obj.text
    else:
        return None

def extract_noun_adjectives(t, doc):
    adjs = []
    adjectives = []
    for child in t.children:
        if child.dep_ == 'appos':
            continue
        if child.dep_ == 'compound':
            continue
        if child.pos_ =='ADJ' or child.tag_ == 'VBN' and child.dep_ in ['conj', 'amod',]:
            
            
            
            advmod = extract_advmod(child, doc)
            measurement = extract_measurements(child, doc)
            prep = extract_prepositions(child, doc)
            nummod = extract_nummod(child, doc)
            amod = extract_amod(child, doc)
            
           
            
            if child.lemma_.lower() in compounds:
                continue
            if child.text.lower() in compounds:
                continue
            elif measurement:
                obj = measurement
                adjs.append(obj)
            elif amod:
                obj = define_position(amod, child, doc)
                adjs.append(obj)
            elif advmod:
                #obj = doc[advmod.i : child.i + 1]
                obj = define_position(advmod, child, doc)
                adjs.append(obj)
            elif prep:
                obj = define_position(prep, child, doc)
                adjs.append(obj)
            elif nummod:
                obj = define_position(nummod, child, doc)
                adjs.append(obj)                
            else:
                obj = child
                adjs.append(obj)
            for grandchild in child.subtree:
                conj = extract_conjunction(grandchild, doc)
                if conj:
                    advmod = extract_advmod(conj, doc)
                    prep = extract_prepositions(conj, doc)
                    nummod = extract_nummod(conj, doc)
                    if advmod:
                        obj = define_position(advmod, conj, doc)
                        adjs.append(obj)
                    elif prep:
                        obj = define_position(prep, conj, doc)
                        adjs.append(obj)
                    elif nummod:
                        obj = define_position(nummod, conj, doc)
                        adjs.append(obj)
                    else:
                        obj = conj
                        adjs.append(obj)            
    for adj in adjs:
        try:
            if adj.pos_ == 'VERB':
                adj_text = adj.text.lower()
            elif adj.root.pos_ == 'VERB':
                adj_text = adj.text.lower()
            else:
                adj_text = adj.lemma_.lower()
        except:
                if type(adj) == str:
                    adj_text = adj.lower()
                else:
                    adj_text = adj.text.lower()
        for adj_split in adj_text.split(','):
            adjectives.append(adj_split.strip())
    return adjectives

def extract_noun_appos(t, doc):
    appos = []
    for child in t.children:
        if child.dep_ == 'appos':
            obj = doc[child.left_edge.i : child.right_edge.i + 1].text.lower()
            for obj_split in obj.split(','):
                appos.append(obj_split.strip())
    return appos

def extract_noun_prep(t, doc):
    preps = []
    for child in t.children:
        if child.dep_ == 'prep':
            obj = doc[child.left_edge.i : child.right_edge.i + 1].text.lower()
            for obj_split in obj.split(','):
                preps.append(obj_split.strip())
    return preps

def check_species(t, species, doc):
    if t.text in species.split():
        return True
    else:
        return False
    
def extract_noun_verbs_ROOT(t, doc):
    relations = []
    objects = []
    if t.dep_ not in ['ROOT', 'nsubj', 'nsubjpass', 'csubj', 'csubjpass']:
        return '', ''
    parent = next((parent for parent in t.ancestors), None)
    if parent and parent.pos_ == 'VERB' or parent and parent.pos_ == 'AUX':
        try:
            prep = extract_verb_prep(parent, doc)
            if prep: 
                relations.append(f'{parent.text} {prep}'), objects.append(extract_verb_pobj(prep, doc).lemma_)
            dobj = extract_verb_dobj(parent, doc)
            if dobj: 
                relations.append(parent.text), objects.append(extract_verb_pobj(dobj, doc).lemma_)
            oprd = extract_verb_orpd(parent, doc)
            if oprd: 
                relations.append(parent.text), objects.append(oprd.text)
            agnt = extract_verb_agnt(parent, doc)
            if agnt: 
                relations.append(f'{parent.text} {agnt}'), objects.append(extract_verb_pobj(agnt, doc).lemma_)
            nmod = extract_verb_nmod(parent, doc)
            if nmod: 
                print(nmod)
            advm = extract_verb_advm(parent, doc)
            if advm: 
                relations.append(parent.text), objects.append(advm.text)
            acomp = extract_verb_acomp(parent, doc)
            if acomp: 
                relations.append(parent.text), objects.append(acomp) # Already text
            if not any((prep, dobj, oprd, agnt, nmod, advm, acomp)): 
                relations.append('are'), objects.append(parent.text)
        except:
            pass
       
    return clean_verbs(relations, objects)

def extract_noun_verbs_NON_ROOT(t, doc):
    relations = []
    objects = []
    if t.dep_ not in ['ROOT', 'nsubj', 'nsubjpass', 'csubj', 'csubjpass']:
        return '', ''
    # Double check
    parent = next((parent for parent in t.ancestors), None)
    if not parent:
        try:
            for child in t.children:
                if child.pos_ == 'VERB' and child.dep_ != 'amod':
                    prep = extract_verb_prep(child, doc)
                    if prep: 
                        noun = extract_verb_pobj(prep, doc)
                        relations.append(f'{child.text} {prep}')
                        objects.append(doc[noun.left_edge.i : noun.right_edge.i + 1].text)
                    dobj = extract_verb_dobj(child, doc)
                    if dobj:
                        relations.append(child.text)
                        objects.append(doc[dobj.left_edge.i : dobj.right_edge.i + 1].text) 
                    oprd = extract_verb_orpd(child, doc)
                    if oprd:
                        oprd_prep = extract_verb_prep(oprd, doc)
                        if oprd_prep:
                            relations.append(f'{child.text} {oprd.text}')
                            objects.append(doc[oprd_prep.left_edge.i : oprd_prep.right_edge.i + 1].text) 
                        else:
                            relations.append(child.text), objects.append(oprd.text)
                    agnt = extract_verb_agnt(child, doc)
                    if agnt:
                        noun = extract_verb_pobj(agnt, doc)
                        relations.append(f'{child.text} {agnt.text}')
                        objects.append(doc[noun.left_edge.i : noun.right_edge.i + 1].text)
                    nmod = extract_verb_nmod(child, doc)
                    if nmod:
                        relations.append('is')
                        objects.append(f'{nmod.text} {child}') 
        except:
            pass
                        
    return clean_verbs(relations, objects)

def extract_verbs_OTHER(t, doc):
    
    '''
    NOT YET IN USE
    '''
    
    relations = []
    objects   = []
    
    empty   = ('', '')
    empties = ([], [])
    
    if extract_noun_verbs_ROOT(t, doc) == empty  and extract_noun_verbs_NON_ROOT(t, doc) == empty:
        
        parent = next((child for child in t.children if child.pos_ == 'VERB'), None)
        prep = extract_verb_prep(parent, doc)
        if prep: 
            relations.append(f'{parent.text} {prep}'), objects.append(extract_verb_pobj(prep, doc).lemma_)
        dobj = extract_verb_dobj(parent, doc)
        if dobj: 
            relations.append(parent.text), objects.append(extract_verb_pobj(prep, doc).lemma_)
        oprd = extract_verb_orpd(parent, doc)
        if oprd: 
            relations.append(parent.text), objects.append(oprd.text)
        agnt = extract_verb_agnt(parent, doc)
        if agnt: 
            relations.append(parent.text), objects.append(agnt.text)
        nmod = extract_verb_nmod(parent, doc)
        if nmod: 
            print(nmod)
        advm = extract_verb_advm(parent, doc)
        if advm: 
            relations.append(parent.text), objects.append(advm.text)
        acomp = extract_verb_acomp(parent, doc)
        if acomp: 
            relations.append(parent.text), objects.append(acomp.text)
        if not any((prep, dobj, oprd, agnt, nmod, advm, acomp)): 
            relations.append('are'), objects.append(parent.text)
            
    return relations, objects


def clean_verbs(relations, objects):
    
    rel = []
    obj = []
    for relation, object_ in zip(relations, objects):
        #rel.append(relation)
        #obj.append(object_.split(',')[0])
        for obj_split in object_.split(','):
            #print(relation, obj_split)
            rel.append(relation.lower().strip())
            obj.append(obj_split.lower().strip())            
            
    return rel, obj
        
def adjective_reverser(doc):
    root = next(t for t in doc if t.dep_ == 'ROOT')
    if root.pos_ == 'ADJ':
        
        span_1 = doc[root.i - 1].text.lower()
        span_2 = root.text.capitalize()
        span_3 = doc[root.i + 1 : ].text
        
        text = f'{span_2} {span_1} {span_3}'
        doc = nlp(text)
        
        print('asd')
    
    return doc        

In [65]:
descriptions = collections.defaultdict(list)

for bird in tqdm_notebook(species_ALL[1:2]):
    idx = species_ALL.index(bird)
    files = glob.glob(species_folder[idx] + '/*')
    for file in files[1:2]:
        with open(file) as f:
            soup = BeautifulSoup(f, 'html.parser')
            dirty_text = soup.get_text(" ", strip=False).replace('\n', '.')
            # Clean and break into sents
            sentences = text_cleaner(dirty_text)

            for idx, sentence in enumerate(sentences[3:4]):
                if classify(sentence, model=model):
                    doc = nlp(sentence)
                    # Init to check empty values
                    descriptions[bird, idx] = []
                    triples = []
                    
                    # EXPERIMENTAL
                    doc = adjective_reverser(doc)
                    
                    # Loop over tokens
                    for t in doc:
                        if t.dep_ == 'compound':
                            continue
                        ### SUBJECTS ###    
                        if t.pos_ == 'NOUN' or t.pos_ == 'PROPN':
                            # Check existance of parts
                            part = check_existance(t, doc, glossary=glossary)
                            if part:
                                # Reconstruct Compounds & Append
                                trait = compound_reconstructor(t, doc)
                                triples.append(('bird', f'has sub part', trait))
                                # NOUN ADJECTIVES
                                adjectives = extract_noun_adjectives(t, doc)
                                for adjective in adjectives:
                                    triples.append((trait, 'is', adjective))
                                # NOUN ROOT VERBS
                                verbs_rel, verbs_obj = extract_noun_verbs_ROOT(t, doc)
                                for rel, obj in zip(verbs_rel, verbs_obj):
                                    triples.append((trait, rel, obj))
                                # NOUN NON ROOT VERBS
                                verbs_rel, verbs_obj = extract_noun_verbs_NON_ROOT(t, doc)
                                for rel, obj in zip(verbs_rel, verbs_obj):
                                    triples.append((trait, rel, obj))
                                # NOUN APPOSITIONAL MODIFIER
                                adjectives = extract_noun_appos(t, doc)
                                for adjective in adjectives:
                                    triples.append((trait, 'is', adjective))
                                # NOUN NUMMODS
                                nummod = extract_dnummod(t, doc)
                                triples.append((trait, 'is', nummod))
                                # NOUN PREPOSITIONAL MODIFIER
                                adjectives = extract_noun_prep(t, doc)
                                for adjective in adjectives:
                                    triples.append((trait, 'is', adjective))                
 

                        #if check_species(t, species, doc):

                    # APPEND
                    descriptions[bird, idx] = [triple for triple in triples if all(triple)]  

                    print(idx, doc)
                    print(descriptions[bird, idx])
                    print('\n')

  0%|          | 0/1 [00:00<?, ?it/s]

0 Throat, breast, and upper abdomen dull white, boldly streaked with dark brown, individual feathers have grayish-white centers and brown margins.
[('bird', 'has sub part', 'throat'), ('throat', 'is', 'dull white'), ('throat', 'is', 'boldly streaked'), ('bird', 'has sub part', 'breast'), ('bird', 'has sub part', 'upper abdomen'), ('bird', 'has sub part', 'individual feather')]




In [63]:
extract_noun_verbs_NON_ROOT(doc[0], doc)

('', '')

In [55]:
displacy.render(doc)

In [16]:
for chunk in doc.noun_chunks:
    print(chunk)

Wings
brown tone
underwing coverts
some individuals


In [22]:
doc = nlp("Wings sooty, with brown tone, underwing coverts black, or in some individuals mottled white.")

In [36]:
def adjective_reverser(doc):
    root = next(t for t in doc if t.dep_ == 'ROOT')
    if root.pos_ == 'ADJ':
        
        span_1 = doc[root.i - 1].text.lower()
        span_2 = root.text.capitalize()
        span_3 = doc[root.i + 1 : ].text
        
        text = f'{span_2} {span_1} {span_3}'
        doc = nlp(text)
    
    return doc
    
adjective_reverser(doc)

Sooty wings , with brown tone, underwing coverts black, or in some individuals mottled white.


In [21]:
displacy.render(nlp("sooty wings, with brown tone, underwing coverts black, or in some individuals mottled white."))

In [20]:
doc.

AttributeError: 'spacy.tokens.doc.Doc' object has no attribute 'head'